# Multimedia Indoor Chemical Screening (MICS) Model 

This notebook can be used to run the MICS Model and analyze results. 

Individual cells below can be run by clicking in the cell and pressing shift + enter. More details regarding the Jupyter Notebook interface can be found using the following link: https://jupyter-notebook.readthedocs.io/en/stable/

## Step 1: Import the MICS Model Package 
The MICS Model directory, which contains the Python code (.py extension), must be in the same parent directory as this Jupyter notebook file. 

In [1]:
# The following statement imports the MICS Model. 
import MICS as ms

# Pandas is a Python library that handles displays in this notebook.
import pandas as pd
# Configure Pandas to not truncate displays. 
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

## Step 2: Define the Model Configuration

The following cell must be run to define the model configuration. This configuration specifies file paths for loading each input parameter file, and for exporting results. 

A default configuration is included below. For example, the file path, `'inputs/default/environment.csv'`, will tell the algorithm to search for the parameter file, `environment.csv`, in the `default` subdirectory of the `inputs` directory. Note that each directory/subdirectory must be separated by a forward slash. 

In [2]:
# Paths can be defined in single or double quotations to the right of each colon. 
configuration = {
	'environment': {
        # Define path to the environment parameters CSV file. 
		'env_parameters': 'inputs/default/environment.csv',
        # Define path to the particle parameters CSV file. 
		'par_parameters': 'inputs/default/particle.csv'
		},
    # Define path to the chemical parameters CSV file. 
	'chemical': 'inputs/sensitivity/chemical-higher-Koa.csv',
	'results': {
        # Define paths and filenames for saving results as CSV files. 
		'chemical_concentrations': 'results/sensitivity/chemical_concentrations.csv',
		'releases_to_outdoors': 'results/sensitivity/releases_to_outdoors.csv',
		'fitted_emissions': 'results/sensitivity/fitted_emission_rates.csv'
	}
}

## Step 3: Define which Compounds to Simulate

The model can be run for all compounds in the chemical parameter input file, or for a subset of compounds. In either case, the user must run one of the following two cells to specify which compounds to simulate. 

Run the following cell to configure all compounds, as indicated by the empty list (square brackets). 

In [3]:
configuration['compounds'] = []

Run the following cell to configure a subset of compounds. Each compound name must match a corresponding name in the chemical parameters input file, INCLUDING SPACES. A common error when defining new names is to accidently include one or more spaces at the end of the compound name. Compound names must be specified in single or double quotations within the list (square brackets), and separated by commas. An example has been provided below for five organophosphate flame retardants. 

In [3]:
configuration['compounds'] = [
    'TCPP',
    'TDCPP',
    'TPhP',
    'TBOEP',
    'EHDPP'
]

## Step 4: Run a Simulation

There are two ways to run a simulation, with the only difference being in how chemical emission rates to indoor air are defined. 

For either function, the user has the option to modify up to two of three key-word arguments (`kwargs`):
- `N_homes` (type = integer): The number of homes for upscaling indoor-to-outdoor chemical release rates to a specified population. Resulting release rates per home will be multiplied by this value. The default value is 1 home, meaning that release rates will NOT be upscaled. 
- `N_persons_per_home` (type = integer): The number of persons per home, used to convert units of indoor-to-outdoor chemical release to a per-person basis. Resulting release rates per home will be divided by this value. The default value is 1 person, meaning that release rates will NOT be converted to a per-person basis, i.e., the units will be [ng/(household-day)] instead of [ng/(person-day)]. *NOTE*: If a user were to change this value, they should make sure that the specified surface area of the textiles compartment in environment.csv corresponds with the number of persons they choose for this key-word argument. The default surface area corresponds to an average of 2.58 persons/household.  
- `export` (type = boolean):  If the value is True, results will be exported as CSV files as specified in model configuration dictionary. If these files already exist, then THEY WILL BE OVERWRITTEN. To avoid accidently overwriting results, the default value is False. *NOTE*: A file must be closed in order to be overwritten, otherwise an error message will be displayed.

*NOTE*: It is appropriate to EITHER upscale the chemical release rates by modifying the default value of `N_homes` OR convert units of chemical release rates to a per-person basis by modifying the default value of `N_persons_per_home`. Both arguments should NOT be modified simultaneously. 

1.) **Run with user-defined emission rates**: The model will use the emission rates [ng/h] specified in the chemical parameters input file. In the absence of information on emission rates, one convenient choice is to set a value of 1 ng/h for each compound. The results could then be interpreted as concentrations and release rates per unit emission rate. Choosing the same emission rate for each compound would enable a user to compare predicted fate across compounds based on differences in physical-chemical property values alone, rather than differences in emission rates. The MICS Model function for this option is `MICS.simulation.run_with_user_defined_emissions`. 

In [4]:
# Define the key-word arguments.
kwargs = {
    'N_homes': 1,  
    'N_persons_per_home': 2.58, 
    'export': True}  # first letter must be capitalized

# Run the simulation and store results. 
# Results are "unpacked" from the tuple into two DataFrames. 
concentrations, release_rates = ms.simulation.run_with_user_defined_emissions(
    configuration, **kwargs)

In [9]:
# Run this cell to update the display of chemical mass concentrations [ng/m3].
concentrations

,air,textiles,PUF,floor,carpet,up_surfs,other_surfs
TCPP,0.017754,381.942468,1.659919e+05,1.772999e+06,316441.184459,4.767329e+06,4.833659e+06
TDCPP,0.010194,615.253184,3.195900e+06,1.752845e+07,405438.786708,6.933427e+07,7.017190e+07
TPhP,0.005723,925.638946,5.157094e+06,3.559300e+07,409464.351568,1.006967e+08,1.013064e+08
TBOEP,0.002626,862.147287,7.185174e+06,3.048799e+07,392249.970751,1.387916e+08,1.298059e+08
EHDPP,0.002621,8295.123270,7.164943e+06,3.287350e+07,392517.399298,1.378002e+08,1.293704e+08


In [7]:
# Run this cell to update the display of chemical release rates to outdoors.
# Units depend on the key-word argument values as detailed above. 
release_rates

,air,wastewater,solid_waste
TCPP,14.3807,9.16624,0.156955
TDCPP,8.25678,14.7423,0.849757
TPhP,4.6357,18.0141,1.30671
TBOEP,2.12676,20.3702,1.49492
EHDPP,2.12301,20.3504,1.51926


2.) **Run with calibrated emission rates**: The model will use an optimization algorithm to find an emission rate for each compound that produces the best fit between observed and predicted concentrations. For this, two additional inputs are required from the chemical parameters file, `concentration` and `compartment`. These inputs denote the measured chemical mass concentration and corresponding model compartment, respectively. The value for `compartment` can be any one of the seven possible model compartments (air, textiles, PUF, floor, carpet, up_surfs, other_surfs). Units of `concentration` must be [ng/m3] for a bulk air sample, or [ng/g] for a dust sample collected from one of the other compartments. The MICS Model function for this option is `MICS.simulation.run_with_fitted_emissions`.

In [14]:
# Define the key-word arguments.
kwargs = {
    'N_homes': 1,
    'N_persons_per_home': 1,
    'export': False}  # first letter must be capitalized

# Run the simulation and store results. 
# Results are "unpacked" from the tuple into two DataFrames and a Series. 
concentrations, release_rates, emission_rates = ms.simulation.run_with_fitted_emissions(
    configuration, **kwargs)

TCPP  calibration success? True
TDCPP  calibration success? True
TPhP  calibration success? True
TBOEP  calibration success? True
EHDPP  calibration success? True


In [15]:
# Run this cell to update the display of chemical mass concentrations [ng/m3].
concentrations

,air,textiles,PUF,floor,carpet,up_surfs,other_surfs
TCPP,2.34,50340.763029,2.187806e+07,2.336847e+08,4.170757e+07,6.283432e+08,6.370857e+08
TDCPP,0.04,2414.281988,1.254086e+07,6.878245e+07,1.590961e+06,2.720709e+08,2.753578e+08
TPhP,4.07,658273.223285,3.667496e+09,2.531216e+10,2.911928e+08,7.161100e+10,7.204462e+10
TBOEP,0.55,180597.223492,1.505105e+09,6.386434e+09,8.216607e+07,2.907319e+10,2.719094e+10
EHDPP,0.04,126594.900891,1.093468e+08,5.016945e+08,5.990351e+06,2.103019e+09,1.974369e+09


In [16]:
# Run this cell to update the display of chemical release rates to outdoors. 
# Units depend on the key-word argument values as detailed above. 
release_rates

,air,wastewater,solid_waste
TCPP,1895.4,1208.13,20.687
TDCPP,32.4,57.8494,3.33448
TPhP,3296.7,12810.8,929.275
TBOEP,445.5,4267.03,313.147
EHDPP,32.4,310.575,23.1859


In [7]:
# Run this cell to update the display of fitted chemical emission rates [ng/h]. 
emission_rates

TCPP     131.801952
TDCPP      3.924046
TPhP     711.155496
TBOEP    209.473748
EHDPP     15.261365
Name: emission_rate, dtype: float64

## Step 5: Change Parameter Values and Re-Run (Optional)
If a user wishes to change one or more input parameter values, re-run a simulation, and compare model results, it is quite simple to do so using this Jupyter notebook interface. The following steps should be taken. 

1. Change the value(s) in the appropriate input CSV file(s). 
2. Save the CSV file(s) and close them. 
2. Restart the kernel (select kernel/restart from the menu bar at the top). 
3. Re-run the appropriate cell(s). *NOTE*: Jupyter saves displays, so to compare model results, one could simply insert a new cell directly above or below the cell where previous results are displayed, and run the same code in the new cell. The outputs will be displayed next to each other in their respective cells, making it easy to compare results side-by-side. Options to insert new cells can be found in the menu bar at the top. 